In [53]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import time
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True,precision=6)
#tf.reset_default_graph()

OP_NONE=-1
OP_BUY=0
OP_SELL=1
ORDER_STATE_OPENED=0
ORDER_STATE_CLOSED=1
POINT=100000
TRADERS_NUM=20
CROSS_PROBALBILITY=0.8
MUTATION_PROBALBILITY=0.05
INITIAL_FOUNDS=1000

def get_datafame(fname):
    df=pd.read_csv(fname,header=None)
    df.columns=['date','time','open','high','low','close','volume']
    df['timestamp']=df['date']+' '+df['time']
    df['timestamp']=df['timestamp'].apply(lambda x:time.mktime(time.strptime(x,'%Y.%m.%d %H:%M'))/time.mktime(time.strptime('2082-02-11 00:00','%Y-%m-%d %H:%M')))
    df['co']=(df['close']-df['open'])*POINT
    return df

def get_data(fname):
    df=get_datafame(fname)
    xi=[];yi=[]
    for i in range(PERIOD,PERIOD+2):
    #for i in range(PERIOD,len(df)):
        xi.append(df['co'][i-PERIOD:i].tolist())
        yi.append(df.iloc[i]['co'])
    xi=np.array(xi)
    return (xi,yi)

class Market(object):
    def __init__(self,spread,series,ntraders):
        self.series=series
        self.ntraders=ntraders
        self.traders=[Trader(self,INITIAL_FOUNDS) for i in range(0,ntraders)]
        self.bar=0
        self.sess=tf.Session()
        self.sess.run(tf.global_variables_initializer())
    
    def move(self):
        for bar in range(0,len(self.series)):
            #print('bar:%f'%(bar))
            self.bar=bar
            self.price=self.series.iloc[bar]['close']
            self.checkOrders()
            #print('before ntraders:%d mean:%f'%(len(self.traders),np.array([trader.accountEquity() for trader in self.traders]).mean()))
            #print(np.array([t.accountEquity() for t in self.traders]))
            #mean=np.array([t.accountEquity() for t in self.traders]).mean()
            #print(mean)
            self.traders=[t for t in self.traders if t.accountEquity()>0]
            print('after ntraders:%d mean:%f'%(len(self.traders),np.array([trader.accountEquity() for trader in self.traders]).mean()))
            if len(self.traders)==0:
                self.crossTraders()
            #self.traders=filter(lambda x:x.accountEquity()>0,self.traders)
            for trader in self.traders:
                trader.makeDecision()
                
    def checkOrders(self):
        #high=int(self.series.iloc[self.bar]['high']*POINT)
        #low=int(self.series.iloc[self.bar]['low']*POINT)
        #if high>low:ticks=np.random.randint(low,high,10)/POINT
        #else: ticks=[low/POINT]
        high=self.series.iloc[self.bar]['high']
        low=self.series.iloc[self.bar]['low']
        ticks=[low,high]
        for price in ticks:
            self.price=price
            for trader in self.traders:
                for order in trader.orders:
                    #print('order:%d'%(order.ticket))
                    if order.op==OP_BUY:
                        if order.stoploss!=0 and price<=order.stoploss:
                            order.close('buy ticket:%d trader:%d stoploss op:%d openprice:%f price:%f takeprofit:%f stoploss:%f'
                                        %(order.ticket,trader.traderNo,order.op,order.openPrice,price,order.takeprofit,order.stoploss))
                        if order.takeprofit!=0 and price>=order.takeprofit:
                            order.close('buy ticket:%d trader:%d takeprofit op:%d openprice:%f price:%f takeprofit:%f stoploss:%f'
                                        %(order.ticket,trader.traderNo,order.op,order.openPrice,price,order.takeprofit,order.stoploss))
                    if order.op==OP_SELL:
                        if order.stoploss!=0 and price>=order.stoploss:
                            order.close('sell ticket:%d trader:%dstoploss op:%d openprice:%f price:%f takeprofit:%f stoploss:%f'
                                        %(order.ticket,trader.traderNo,order.op,order.openPrice,price,order.takeprofit,order.stoploss))
                        if order.takeprofit!=0 and price<=order.takeprofit:
                            order.close('sell ticket:%d trader:%d takeprofit op:%d openprice:%f price:%f takeprofit:%f stoploss:%f'
                                        %(order.ticket,trader.traderNo,order.op,order.openPrice,price,order.takeprofit,order.stoploss))
                trader.orders=[order for order in trader.orders if order.state==ORDER_STATE_OPENED]
                #trader.orders=filter(lambda x:x.state==ORDER_STATE_OPENED,trader.orders)
    
    def crossTraders(self):
        if len(self.traders)==0:
            self.traders=[Trader(self,INITIAL_FOUNDS) for i in range(0,self.ntraders)]
            return
        while(len(self.traders)<self.ntraders):
            male=self.traders[np.random.randint(0,len(self.traders))]
            female=self.traders[np.random.randint(0,len(self.traders))]
            child=male.cross(female)
            self.traders.append(child)
        for trader in self.traders:
            trader.orders=[]
            trader.balance=INITIAL_FOUNDS
    def report(self):
        index=0
        for trader in sorted(self.traders, key=lambda x:x.profit, reverse=True):
            print(trader.profit)
            index+=1
            if index>3:break
traderNo=0
class Trader(object):
    def __init__(self,market,fouds):
        global traderNo;
        self.traderNo=traderNo
        traderNo+=1
        hidden_num=30
        self.market=market
        self.balance=fouds
        self.equity=fouds
        self.margin=0
        self.profit=0
        self.orders=[]
        self.period=np.random.randint(5,200)
        #self.weights_in=tf.Variable(tf.truncated_normal(shape=[hidden_num,self.period],stddev=1.0/tf.sqrt(self.period)))
        #self.weights_hidden
        #self.weights_in=np.random.randn(hidden_num,self.period)
        #self.weights_hidden=np.random.randn(hidden_num)
        #self.bias_hidden=np.random.randn(hidden_num,1)
        #self.weights_out=np.random.randn(3,hidden_num)
        stddev=1.0/tf.sqrt(tf.cast(self.period,tf.float32))
        self.weights_in=tf.Variable(tf.truncated_normal(shape=[hidden_num,self.period],stddev=stddev))
        self.weights_hidden=tf.Variable(tf.truncated_normal(shape=[hidden_num],stddev=stddev))
        self.bias_hidden=tf.Variable(tf.truncated_normal(shape=[hidden_num,1]))
        self.weights_out=tf.Variable(tf.truncated_normal(shape=[3,hidden_num]))
        self.x=tf.placeholder(shape=[self.period,1],dtype=tf.float32)
        
        y=tf.matmul(self.weights_in,self.x)
        y=tf.add(y,self.bias_hidden)
        #print('add')
        #print(self.market.sess.run(x))
        y=tf.nn.sigmoid(y)
        y=tf.matmul(self.weights_out,y)
        #print('matmul')
        #print(self.market.sess.run(x))
        y=tf.reshape(y,(3,))
        y=tf.nn.softmax(y)
        y=tf.argmax(y)
        self.y=y
    
    def pretrain(self):
        pass;
        
    def sigmoid(self,x):  
        return 1.0 / (1 + np.exp(-x))
    
    def softmax(self,x):
        return np.exp(x)/np.sum(np.exp(x),axis=0)
        
    
    def makeDecision(self):
        if self.market.bar<self.period:
            return
        xi=np.array(self.market.series[self.market.bar-self.period:self.market.bar]['close'])[:,np.newaxis]
        op=self.market.sess.run(self.y,feed_dict={self.x:xi})-1
        #print('op: %d'%(op))
        if op==OP_NONE:return
        if len(self.orders)>0:return
        lots=1
        takeprofit=0.001
        stoploss=0.001
        #lots=round(np.clip(o2[1],0.1,0.5),2)
        #takeprofit=round(np.clip(o2[2],0,500.0/POINT),5)
        #stoploss=round(np.clip(o2[3],0,500.0/POINT),5)
        #print('op:%d lots:%f takeprofit:%f stoploss:%f'%(op,lots,takeprofit,stoploss))
        if lots==0: return
        if op==OP_BUY:
            takeprofit=self.market.price+takeprofit
            stoploss=self.market.price-stoploss
            self.buy(lots,takeprofit,stoploss)
        if op==OP_SELL:
            takeprofit=self.market.price-takeprofit
            stoploss=self.market.price+stoploss
            self.sell(lots,takeprofit,stoploss)
        
    def buy(self,lots,takeprofit,stoploss):
        order=Order(self.market,self)
        order.open(OP_BUY,lots,takeprofit,stoploss)
        #self.orders.append(order)
    def sell(self,lots,takeprofit,stoploss):
        order=Order(self.market,self)
        order.open(OP_SELL,lots,takeprofit,stoploss)
        #self.orders.append(order)
    def accountBalance(self):
        return self.balance
    def accountProfit(self):
        return sum([order.orderProfit() for order in self.orders if order.state==ORDER_STATE_OPENED])
        #return sum(filter(lambda x:x.state==ORDER_STATE_OPENED,self.orders))
    def accountEquity(self):
        return self.balance+self.accountProfit()
    def orders():
        return self.orders
    def cross(self,female):
        child=Trader(self.market,INITIAL_FOUNDS)
        child.period=self.period
        self.market.sess.run(tf.assign(child.weights_in,self.weights_in))
        self.market.sess.run(
            tf.assign(child.weights_hidden,(CROSS_PROBALBILITY*self.weights_hidden+(1-CROSS_PROBALBILITY)*female.weights_hidden)))
        child.mutate()
        return child
    def mutate(self):
        self.market.sess.run(tf.assign(self.weights_hidden,tf.nn.dropout(self.weights_hidden,1-MUTATION_PROBALBILITY)))
    #def dropout(self,weights,prob):
    #    mask=np.random.random(weights.shape)
    #    mask=mask<(1-prob)
    #    return weights*mask
    
ticket=0
class Order(object):
    def __init__(self,market,trader):
        global ticket
        self.market=market
        self.trader=trader
        self.ticket=ticket
        ticket+=1
        
    def open(self,op,lots,takeprofit,stoploss):
        self.op=op
        self.openPrice=self.market.price
        self.lots=lots
        self.takeprofit=takeprofit
        self.stoploss=stoploss
        self.trader.orders.append(self)
        self.state=ORDER_STATE_OPENED
        #market.orders.append(self)
    def orderProfit(self):
        if self.op==OP_SELL:
            return self.lots*(self.openPrice-self.market.price)*POINT
        if self.op==OP_BUY:
            return self.lots*(self.market.price-self.openPrice)*POINT
    def close(self,notes):
        profit=self.orderProfit()
        self.trader.balance+=profit
        self.state=ORDER_STATE_CLOSED
        #print(notes)
        #print('op:%d openPrice:%f price:%f profit:%f takeprofit:%f stoploss:%f'%(self.op,self.openPrice,self.trader.market.price,profit
        #                                                                         ,self.takeprofit,self.stoploss))
        #self.traders.orders.remove(self)
        #self.market.orders.remove(self)

def main():
    market=Market(2,get_datafame('test.csv'),TRADERS_NUM)
    EPOCH=1
    for i in range(0,EPOCH):
        market.move()
        market.crossTraders()
        market.report()


In [54]:
import cProfile
cProfile.run('main()')

after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 mean:1000.000000
after ntraders:20 me

after ntraders:20 mean:1024.200000
after ntraders:20 mean:1024.950000
after ntraders:20 mean:1023.950000
after ntraders:20 mean:1023.700000
after ntraders:20 mean:1024.450000
after ntraders:20 mean:1023.950000
after ntraders:20 mean:1023.450000
after ntraders:20 mean:1023.200000
after ntraders:20 mean:1022.200000
after ntraders:20 mean:1021.200000
after ntraders:20 mean:1021.700000
after ntraders:20 mean:1020.950000
after ntraders:20 mean:1016.700000
after ntraders:20 mean:1014.700000
after ntraders:20 mean:1015.950000
after ntraders:20 mean:1016.450000
after ntraders:20 mean:1018.450000
after ntraders:20 mean:1019.950000
after ntraders:20 mean:1019.950000
after ntraders:20 mean:1018.950000
after ntraders:20 mean:1017.950000
after ntraders:20 mean:1018.450000
after ntraders:20 mean:1016.200000
after ntraders:20 mean:1014.450000
after ntraders:20 mean:1014.450000
after ntraders:20 mean:1014.450000
after ntraders:20 mean:1014.700000
after ntraders:20 mean:1016.700000
after ntraders:20 me

after ntraders:20 mean:1047.050000
after ntraders:20 mean:1043.550000
after ntraders:20 mean:1043.800000
after ntraders:20 mean:1042.050000
after ntraders:20 mean:1044.550000
after ntraders:20 mean:1044.800000
after ntraders:20 mean:1048.550000
after ntraders:20 mean:1048.300000
after ntraders:20 mean:1048.800000
after ntraders:20 mean:1049.050000
after ntraders:20 mean:1046.050000
after ntraders:20 mean:1046.550000
after ntraders:20 mean:1047.800000
after ntraders:20 mean:1048.050000
after ntraders:20 mean:1044.300000
after ntraders:20 mean:1045.550000
after ntraders:20 mean:1042.800000
after ntraders:20 mean:1042.300000
after ntraders:20 mean:1042.550000
after ntraders:20 mean:1043.800000
after ntraders:20 mean:1043.550000
after ntraders:20 mean:1042.800000
after ntraders:20 mean:1044.300000
after ntraders:20 mean:1048.300000
after ntraders:20 mean:1048.300000
after ntraders:20 mean:1050.300000
after ntraders:20 mean:1052.550000
after ntraders:20 mean:1053.050000
after ntraders:20 me

after ntraders:20 mean:1033.500000
after ntraders:20 mean:1028.500000
after ntraders:20 mean:1028.500000
after ntraders:20 mean:1027.750000
after ntraders:20 mean:1028.500000
after ntraders:20 mean:1025.750000
after ntraders:20 mean:1024.250000
after ntraders:20 mean:1023.000000
after ntraders:20 mean:1022.250000
after ntraders:20 mean:1023.750000
after ntraders:20 mean:1026.500000
after ntraders:20 mean:1028.500000
after ntraders:20 mean:1030.500000
after ntraders:20 mean:1031.750000
after ntraders:20 mean:1028.000000
after ntraders:20 mean:1029.250000
after ntraders:20 mean:1029.250000
after ntraders:20 mean:1029.500000
after ntraders:20 mean:1029.750000
after ntraders:20 mean:1024.750000
after ntraders:20 mean:1024.250000
after ntraders:20 mean:1027.750000
after ntraders:20 mean:1029.750000
after ntraders:20 mean:1031.500000
after ntraders:20 mean:1030.500000
after ntraders:20 mean:1029.250000
after ntraders:20 mean:1029.500000
after ntraders:20 mean:1029.250000
after ntraders:20 me

after ntraders:20 mean:1015.650000
after ntraders:20 mean:1013.150000
after ntraders:20 mean:1014.150000
after ntraders:20 mean:1011.650000
after ntraders:20 mean:1017.150000
after ntraders:20 mean:1018.400000
after ntraders:20 mean:1010.900000
after ntraders:20 mean:1010.650000
after ntraders:20 mean:1011.150000
after ntraders:20 mean:999.650000
after ntraders:20 mean:998.900000
after ntraders:20 mean:1002.150000
after ntraders:20 mean:1002.650000
after ntraders:20 mean:995.900000
after ntraders:20 mean:993.150000
after ntraders:20 mean:993.150000
after ntraders:20 mean:999.400000
after ntraders:20 mean:1000.650000
after ntraders:20 mean:997.650000
after ntraders:20 mean:997.150000
after ntraders:20 mean:994.650000
after ntraders:20 mean:993.650000
after ntraders:20 mean:991.900000
after ntraders:20 mean:990.400000
after ntraders:20 mean:993.400000
after ntraders:20 mean:990.900000
after ntraders:20 mean:992.900000
after ntraders:20 mean:992.150000
after ntraders:20 mean:990.150000
af

    19102    0.017    0.000    0.028    0.000 c_api_util.py:33(__del__)
     3000    0.024    0.000    0.230    0.000 cast.py:1049(find_common_type)
     6000    0.005    0.000    0.014    0.000 cast.py:1074(<genexpr>)
    18000    0.006    0.000    0.008    0.000 cast.py:1077(<genexpr>)
     6000    0.003    0.000    0.010    0.000 cast.py:1081(<genexpr>)
     6000    0.002    0.000    0.007    0.000 cast.py:1083(<genexpr>)
    18000    0.007    0.000    0.054    0.000 cast.py:1088(<genexpr>)
     3012    0.005    0.000    0.005    0.000 cast.py:826(maybe_castable)
        5    0.000    0.000    0.000    0.000 cast.py:838(maybe_infer_to_datetimelike)
     3005    0.019    0.000    0.057    0.000 cast.py:935(maybe_cast_to_datetime)
        9    0.000    0.000    0.000    0.000 common.py:1009(is_datetime64_any_dtype)
        1    0.000    0.000    0.000    0.000 common.py:102(_expand_user)
        3    0.000    0.000    0.000    0.000 common.py:1136(is_datetime_or_timedelta_dtype)
     

        4    0.000    0.000    0.000    0.000 parse.py:98(_noop)
        1    0.000    0.000    0.001    0.001 parsers.py:1047(_make_engine)
        1    0.000    0.000    0.003    0.003 parsers.py:1064(read)
        1    0.000    0.000    0.000    0.000 parsers.py:1095(_create_index)
        3    0.000    0.000    0.000    0.000 parsers.py:1109(_is_index_col)
        1    0.000    0.000    0.000    0.000 parsers.py:1158(_validate_usecols_arg)
        1    0.000    0.000    0.000    0.000 parsers.py:1198(_validate_parse_dates_arg)
        1    0.000    0.000    0.000    0.000 parsers.py:1221(__init__)
        1    0.000    0.000    0.000    0.000 parsers.py:1294(_has_complex_date_col)
        1    0.000    0.000    0.000    0.000 parsers.py:1374(_maybe_dedup_names)
        2    0.000    0.000    0.000    0.000 parsers.py:1398(_maybe_make_multi_index_columns)
        2    0.000    0.000    0.000    0.000 parsers.py:1402(<listcomp>)
        1    0.000    0.000    0.000    0.000 parsers.p